# Análise Exploratória da Tabela de Requisição de Materiais



Este notebook apresenta a análise exploratória de dados da tabela de requisição de materiais. o principal intuito desta análise é entender melhor os dados relacionados a materiais e a partir disto gerar hipóteses.

Nesse sentido, aqui serão examinadas as principais características de um conjunto de dados, identificando padrões, tendências, outliers e possíveis relações entre variáveis. Utilizando estatísticas descritivas, gráficos e visualizações, a AED ajudará a entender a estrutura dos dados, verificar hipóteses e orientar etapas futuras, como modelagem ou limpeza. Essa análise é essencial para fazer escolhas informadas sobre quais métodos e técnicas aplicar no restante do projeto.

O notebook apresenta as seguintes seções:
1. Instalações
2. Importação da Base
3. Início das Análises 
4. Considerações Finais



# 1. Instalações 

In [ ]:
%pip install boto3

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import boto3
import pandas as pd
import pyarrow.parquet as pq


# 2. Importação da Base

In [ ]:
database = pd.read_csv('TABELA_BIG_DATA_REQ_MAT_202410111115.csv')


# 3. Início das Análises 


Ao visualizar as primeiras linhas da tabela é possível perceber que muitas colunas apresentam valores nulos como 'NaN', pensando nisso, essas colunas serão analisadas de forma mais aprofundada para entender se realmente uma parte considerável dos seus valores são nulos.

In [ ]:
database.head()

In [ ]:
database.columns

In [ ]:
pd.set_option('display.max_columns', None)
database.head()

In [ ]:
database.describe()

In [ ]:
print(f"Comprimento da base: {len(database)}")

for column in database:
    print(f"Qtd. de NaN na coluna '{column}': {database[column].isna().sum()}")


A partir da célula acima é possível ver que as colunas: "TIPOREQMATCOD", "ITREQMATOBS", "REQMATDATACANC", "REQMATUSUCODCANC", "MOTCANCCODESTR", "ITREQMATCANCDATA", "ITREQMATCANCUSUCOD", "REQMATTEXTOCANC1", "REQMATTEXTOCANC2", "REQMATTEXTOCANC3", "REQMATTEXTO2" e "REQMATTEXTO3" apresentam todos ou quase todos os valores nulos. Por isso, para realizar esta análise, a princípio, serão removidas todas as colunas que apresentam mais de 100.000 valores nulos.

In [ ]:
database_wout_nan = database
for column in database_wout_nan:
    if database_wout_nan[column].isna().sum() > 100000:
        database_wout_nan =  database_wout_nan.drop(column, axis=1)
    else:
        continue
    
print(database_wout_nan.shape)
database_wout_nan.head()


In [ ]:
for column in database_wout_nan:
    print(f" '{column}':  {len(database_wout_nan[column].unique())}")


No output gerado acima é possível perceber que a coluna "DT_CARGA" apresenta apenas uma data. No momento foi levantada uma hipótese para justificar tal fato:
- Esta coluna apresenta a data de realização da carga da base de dados para a utilização no projeto, o que justificaria a existência de apenas um valor na coluna e a existência da coluna de "REQMATDATA" que possui valores diferentes.

Além disto, a coluna "REQMATNUM" apresenta apenas 70063 linhas, frente às 147006 linhas totais na tabela. Isto mostra que muitos valores se repetem. Acredita-se que a coluna em questão apresenta o ID da requisição de materiais, a princípio acreditou-se que este ID estaria repetido devido a duplicatas, no entanto ao analisar por exemplo as requisições com o ID '408618', percebe-se que os valores nas colunas "ITREQMATQTD", "ITREQMATQTDCALC", "ITREQMATUNIDMEDCOD", "ITREQMATQTDATEND" e "ITREQMATQTDATENDCALC" são diferentes em cada linha. Ainda não se fez possível entender completamente o porquê da questão, mas tendo em vista que os valores nas outras colunas são exatamente inguais em todas as linhas, acredita-se que:
- A coluna 'REQMATNUM' apresenta o ID da requisiçã de materiais e que em uma única requisição podem ser solicitados diversos materiais em quantidades diferentes e com unidades de medida diversas.

In [ ]:
database_wout_nan[database_wout_nan['REQMATNUM'] == 408618]

Neste ponto foram analisadas as colunas com valores numéricos citadas no texto acima, as quais são as únicas features que apresentam valores diferentes mesmo qeu possuam o mesmo ID de requisição. Assim, foram encintrados outliers quem devido à sua distância do comportamento padrão dos dados, foram removidos a fim de diminuir os ruídos da análise.

In [ ]:
database_wout_nan['ITREQMATQTDATENDCALC'].plot(kind='box', figsize=(7,3), vert=False)
plt.show()
database_wout_nan['ITREQMATQTDATEND'].plot(kind='box', figsize=(7,3), vert=False)
plt.show()
database_wout_nan['ITREQMATQTDCALC'].plot(kind='box', figsize=(7,3), vert=False)
plt.show()
database_wout_nan['ITREQMATQTD'].plot(kind='box', figsize=(7,3), vert=False)
plt.show()

In [ ]:
if database_wout_nan['ITREQMATQTDCALC'].equals(database_wout_nan['ITREQMATQTD']):
    print("Colunas iguais")  
else:
    print("'ITREQMATQTDCALC' e 'ITREQMATQTD' são diferentes")  


In [ ]:
if database_wout_nan['ITREQMATQTDATEND'].equals(database_wout_nan['ITREQMATQTDATENDCALC']):
    print("Colunas iguais")  
else:
    print("'ITREQMATQTDATEND' e 'ITREQMATQTDATENDCALC' são diferentes")  

In [ ]:

def remove_outliers(df_cleaned, threshold=3):
    for column in df_cleaned.columns:
        if np.issubdtype(df_cleaned[column].dtype, np.number):  
            mean = np.mean(df_cleaned[column])
            std_deviation = np.std(df_cleaned[column])
            z_scores = (df_cleaned[column] - mean) / std_deviation
            
            outliers = abs(z_scores) > threshold
            
            df_cleaned = df_cleaned[~outliers]
    
    return df_cleaned

database_wout_nan_outliers = remove_outliers(database_wout_nan, 3)     

print(f'Shape com outliers: {database_wout_nan.shape}')
print(f'Shape sem outliers: {database_wout_nan_outliers.shape}')  
database_wout_nan_outliers.head()

In [ ]:
database_wout_nan_outliers['REQMATSTAT'].value_counts().plot(kind='bar', figsize=(14,6), title='Distribuição de Status de Requisição de Material')

A partir dos gráficos apresentados a seguir seria interessante descobrir qual é a descrição de cada um dos 6 tipos de Status das requisições, dando mais enfoque àqueles de entendimento não tão claro, como: Pendente, Separada e Aberta. Isto pois seria interessante para as análises saber, por exemplo, qual a diferença de uma requisição Aberta e Pendente, dado que a partir do momento que uma requisição foi aberta já está pendente.

In [ ]:

top_categories = ['Atendida Total', 'Atendida Parcial', 'Cancelada']
data = database_wout_nan_outliers['REQMATSTAT'].apply(lambda x: x if x in top_categories else 'Outras')

category_counts = data.value_counts(normalize=True) * 100  

plt.figure(figsize=(8, 8))
category_counts.plot(
    kind='pie', 
    autopct='%1.1f%%',  
    labels=category_counts.index, 
    colors=['skyblue', 'lightgreen', 'salmon', 'grey'], 
    startangle=90 ,
    figsize=(12,12) 
)
plt.ylabel('')  
plt.title('Distribuição de Status de Requisição de Material')
plt.show()


In [ ]:
database_wout_nan_outliers['REQMATORIG'].value_counts().plot(kind='pie', figsize=(14,6), autopct='%1.1f%%', title='Distribuição de Origem de Requisição de Material')


In [ ]:
database_wout_nan.dtypes

In [ ]:
categorical_columns = []

for column in database_wout_nan:
    if database_wout_nan[column].dtypes == object:
        categorical_columns.append(column)
    else:
        continue
    
categorical_columns

Para facilitar a análise e geração de insights, abaixo é realizada a transformação de todas variaveis categóricas em variaveis numéricas através do Label Encoder.

In [ ]:
label_encoder = LabelEncoder()

for column in categorical_columns:
    database_wout_nan_outliers[column] = label_encoder.fit_transform(database_wout_nan_outliers[column])

database_wout_nan_outliers.head()


In [ ]:
for column in database_wout_nan_outliers:
    print(f"Qtd. de NaN na coluna '{column}': {database_wout_nan_outliers[column].isna().sum()}")

In [ ]:
database_wout_nan_outliers.dropna(inplace=True)
for column in database_wout_nan_outliers:
    print(f"Qtd. de NaN na coluna '{column}': {database_wout_nan_outliers[column].isna().sum()}")

In [ ]:
database_wout_nan_outliers.head()

Devido ao grande número de features na tabela, uma matriz de correlação dificilmente trará contribuições significativas à análise e que possam ser facilmente visualizadas, para provar este fato, tal matriz foi construída abaixo. 

E,bora possua um baixo nível de legibilidade, ainda assim a matriz traz algumas correlações que podem ser interessantes. 

In [ ]:
correlation_matrix = database_wout_nan_outliers.corr()

plt.figure(figsize=(20, 20))

sns.heatmap(
    correlation_matrix, 
    annot=True,        
    cmap="coolwarm",   
    fmt=".2f",        
    linewidths=0.5,  
    square=True,      
    cbar_kws={"shrink": 0.8}  
)

plt.title("Matriz de Correlação")
plt.show()

In [ ]:
scaler = StandardScaler()
X_database = scaler.fit_transform(database_wout_nan_outliers)

pca = PCA()
pca.fit(X_database)

explained_variance_ratio = pca.explained_variance_ratio_

plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o', linestyle='--')
plt.title('Explained Variance by Principal Components (Elbow Plot)')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.grid()
plt.show()

cumulative_variance = np.cumsum(explained_variance_ratio)

plt.figure(figsize=(10, 6))
plt.plot(np.arange(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.axhline(y=0.90, color='r', linestyle='--', label='90% Variance Explained')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% Variance Explained')
plt.title('Cumulative Explained Variance by Principal Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid()
plt.legend()
plt.show()

In [ ]:
pca = PCA(n_components=5)
X_database_pca = pca.fit_transform(X_database)

print(f'Variância explicada: {pca.explained_variance_ratio_}')


In [ ]:
plt.figure(figsize=(8, 6))

plt.scatter(X_database_pca[:, 0], X_database_pca[:, 1], alpha=0.7, c='blue', s=50)

plt.title('PCA - First Two Principal Components')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)

plt.show()

In [ ]:
database_wout_nan_outliers.to_parquet('../../Datalake/req_materiais.parquet')


# 4. Considerações Finais

A tabela não possibilitou a extração de muitos insights. Acredita-se que para uma melhor geração destes seja necessário um  dicionário que mostre o real significado de cada uma das colunas e possibilite entendê-las. Isto, pois muitas colunas apresentam nomes parecidos compostos por abreviações. Caso seja possível entender o que cada uma destas de fato significa, se fará possível olhar com mais intencionalidade para os dados apresentados por estas e, por conseguinte, gerar hipóteses efetivas e insights.